In [19]:
"""
Created on 26.09.2023
"""

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


def FrankeFunction(x, y):
    term1 = 0.75 * np.exp(-(0.25 * (9 * x - 2)**2) - 0.25 * ((9 * y - 2)**2))
    term2 = 0.75 * np.exp(-((9 * x + 1)**2) / 49.0 - 0.1 * (9 * y + 1))
    term3 = 0.5 * np.exp(-(9 * x - 7)**2 / 4.0 - 0.25 * ((9 * y - 3)**2))
    term4 = -0.2 * np.exp(-(9 * x - 4)**2 - (9 * y - 7)**2)
    return term1 + term2 + term3 + term4


def create_X(x, y, n):
    """Returns the design matrix X from coordinates x and y with n polynomial degrees."""
    if len(x.shape) > 1:
        x = np.ravel(x)
        y = np.ravel(y)

    N = len(x)
    l = int((n + 1) * (n + 2) / 2)  # Number of elements in beta
    X = np.ones((N, l))

    for i in range(1, n + 1):
        q = int(i * (i + 1) / 2)
        for k in range(i + 1):
            X[:, q + k] = (x**(i - k)) * (y**k)

    return X


def MSE(y, y_tilde):
    """Returns the mean squared error of the two arrays."""
    return np.mean((y - y_tilde)**2)


def R2_score(y, y_tilde):
    """Returns the R2 score of the two arrays."""
    return 1 - np.sum((y - y_tilde)**2) / np.sum((y - np.mean(y))**2)


def MSE_Ridge_beta_R2(X_train, X_test, y_train, y_test, lamb): 
    """MSE of Ridge"""
    TestMSE = np.zeros(len(lamb))
    TrainMSE = np.zeros(len(lamb))
    R2_val = np.zeros(len(lamb))
    beta = np.zeros(len(lamb))
    for i in range(len(lamb)):
        print(np.shape(X_train.T @ y_train))
        print(np.shape(X_train.T), np.shape(y_train))
        
        beta_Rid_train = np.linalg.pinv(X_train.T @ X_train + lamb[i] * np.identity(degree)) @ X_train.T @ y_train
        y_tilde = X_train @ beta_Rid_train
        TrainMSE[i] = MSE(y_train, y_tilde)
        
        beta_Rid_test = np.linalg.pinv(X_test.T @ X_test + lamb[i] * np.identity(degree)) @ X_test.T @ y_test
        y_predict = X_test @ beta_Rid_test
        TestMSE[i] = MSE(y_test, y_predict)
        
        # MSE, beta and R2 values
        train_mse[degree] = TrainMSE_Rid 
        test_mse[degree] = TestMSE_Rid
        beta[i] = beta_Rid_train
        R2_val[i] = R2_score(y_test, y_tilde)
    return TrainMSE, TestMSE, beta, R2


# PARAMETERS
n = 1000  # number of data points
maxdegree = 6  # max polynomial degree for plotting
lmbda_vals = np.asarray([0.0001, 0.001, 0.01, 0.1, 1])

# Figure output directory
RESULTS_DIR = Path("../results").resolve()
FIGURES_DIR = RESULTS_DIR / "figures"

# Create them if they dont exist
if not RESULTS_DIR.exists():
    RESULTS_DIR.mkdir()

if not FIGURES_DIR.exists():
    FIGURES_DIR.mkdir()

# Create data set
np.random.seed(2023)
x = np.sort(np.random.uniform(0, 1, n))
y = np.sort(np.random.uniform(0, 1, n))

# Get franke function with noise
noise = np.random.normal(0, 0.1, x.shape)
z = FrankeFunction(x, y) + noise
scaler = StandardScaler()

# Make lists which will contain plot data
degrees = np.arange(0, maxdegree, 1)
train_mse = np.empty(degrees.shape)
test_mse = np.empty(degrees.shape)
Beta = np.empty(degrees.shape)
R2 = np.empty(degrees.shape)


# Iterate through all the different polynomial degrees
for degree in degrees:
    # Create design matrix X
    X = create_X(x, y, n=degree)  # intercept included
    
    # Split in training and test data
    X_train, X_test, z_train, z_test = train_test_split(X, z, test_size=0.2)
    # Scale data
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Perform ridge regression
    TrainMSE_Rid, TestMSE_Rid, beta, R2_val = MSE_Ridge_beta_R2(X_train_scaled, X_test_scaled, z_train, z_test, lmbda_vals)
    train_mse[degree] = TrainMSE_Rid
    test_mse[degree] = TestMSE_Rid
    Beta[degree] = beta
    R2[degree] = R2_val
    

plt.figure(figsize=(10,8))
plt.title("MSE-value")
for degree in degrees:
    plt.subplot(3, 2, degree + 1)
    plt.plot(lambda_vals, train_MSE[degree], "r-", label=f"train, deg={degree}")
    plt.plot(lambda_vals, test_MSE[degree], "r", label=f"test, deg={degree}")
    plt.legend()
    plt.xlabel(r"$\lambda$")
    plt.ylabel("MSE")
plt.show()

plt.figure(figsize=(10,8))
plt.title("Beta-value")
for degree in degrees:
    plt.subplot(3, 2, degree + 1)
    plt.plot(lambda_vals, Beta[degree], "b", label=f"Beta, deg={degree}")
    plt.legend()
    plt.xlabel(r"$\lambda$")
    plt.ylabel(r"$\beta$")
plt.show()

plt.figure(figsize=(10,8))
plt.title("R2-score")
for degree in degrees:
    plt.subplot(3, 2, degree + 1)
    plt.plot(lambda_vals, R2[degree], "g", label=f"R2, deg={degree}")
    plt.legend()
    plt.xlabel(r"$\lambda$")
    plt.ylabel("R2")
plt.show()


(1,)
(1, 800) (800,)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 0)